# smolagents with AzureOpenAI model


In [ ]:
%pip install opendatasets openai unstructured[pdf] gradio langchain-openai aperturedb pandas langchain-community smolagents 'smolagents[litellm]' arxiv --upgrade --quiet

In [ ]:
!sudo apt-get update
!apt-get install poppler-utils

In [1]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /home/sg/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [2]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/sg/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [ ]:
!pip install tesseract

In [ ]:
!apt install tesseract-ocr

In [ ]:
! adb config create --active --from-json 

In [ ]:
adb utils execute summary

In [1]:
import os
import json
import arxiv
import requests
import pandas as pd
import opendatasets as od
from langchain_core.documents import Document
from unstructured.partition.auto import partition
#from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain.chains import (
    StuffDocumentsChain, LLMChain
)
from langchain.schema import HumanMessage, AIMessage
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.callbacks.manager import (
    trace_as_chain_group,
)
import gradio as gr

In [4]:
dataset = 'https://www.kaggle.com/datasets/Cornell-University/arxiv'
od.download(dataset)

Dataset URL: https://www.kaggle.com/datasets/Cornell-University/arxiv


In [2]:
def fetch_paper_details(arxiv_id):
    paper = next(arxiv.Client().results(arxiv.Search(id_list=[arxiv_id])))
    paper.download_pdf( filename=f"{arxiv_id}.pdf")
    return partition(f"{arxiv_id}.pdf")

In [ ]:
sudo apt update
sudo apt install -y libgl1

In [ ]:
papers = []
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=5000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

sample = 5 # Arxiv has over 1.7M articles, using 20 for our application

# Open the JSON file and process entries
with open("arxiv/arxiv-metadata-oai-snapshot.json", "r") as file:
    for _ in range(sample):
        line = file.readline()
        data = json.loads(line)

        # Extract relevant fields
        arxiv_id = data.get("id", "")

        # Add paper details by downloading and parsing the paper
        paper_details = "".join(
            text if isinstance((text := element.text), str)
            else "".join(str(part) for part in text) if isinstance(text, (list, tuple))
            else str(text)
            for element in fetch_paper_details(arxiv_id)
        )
        print(type(paper_details))
        # Use LangChain's splitter to divide paper details into chunks
        chunks = text_splitter.create_documents([paper_details])
        print(len(chunks))
        # Create a Document for each chunk
        for idx, chunk in enumerate(chunks):
            print(chunk,type(chunk))
            document_id = f"{arxiv_id}_{idx + 1}"  # Unique ID for each chunk
            document = Document(
                page_content=chunk.page_content,
                id=document_id,
                metadata={
                    'title': data.get("title",""),
                    'authors': data.get("authors", ""),
                    'submitter': data.get("submitter", ""),
                    'abstract': data.get("abstract", ""),
                    'paper_content': chunk.page_content
                }
            )
            papers.append(document)

print("Processing complete. Papers saved to processed_papers.json.")

In [6]:
from dotenv import load_dotenv
load_dotenv()
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
KAGGLE_TOKEN = os.getenv("KAGGLE_TOKEN")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_API_VERSION = os.getenv("AZURE_API_VERSION")
deployment = os.getenv("AZURE_DEPLOYMENT")

In [ ]:
print(AZURE_OPENAI_ENDPOINT,  AZURE_API_VERSION ,deployment)

In [ ]:
python -c "import openai; print(openai.__version__)"

In [ ]:
# #Below step create ApertureDB and use the connection string in .env file
# KAGGLE_TOKEN=""
# KAGGLE_USERNAME=

# AZURE_DEPLOYMENT="gpt-4"
# AZURE_API_VERSION="2024-08-01-preview"
# AZURE_OPENAI_ENDPOINT="https://<>.openai.azure.com/"
# AZURE_OPENAI_API_KEY=""

# LANGCHAIN_TRACING_V2=true
# LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
# LANGCHAIN_API_KEY=""
# LANGCHAIN_PROJECT=""

# APERTUREDB_JSON={"host":"<>.farm0003.cloud.aperturedata.io","port":55555,"username":"","password":"","name":"","use_ssl":true,"use_rest":false,"use_keepalive":true,"retry_interval_seconds":1,"retry_max_attempts":3}


In [11]:
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain_community.vectorstores import ApertureDB

embeddings = AzureOpenAIEmbeddings(model="text-embedding-3-large")
vector_db = ApertureDB.from_documents(papers, embeddings)

In [34]:
import os
from langchain.embeddings import AzureOpenAIEmbeddings
from openai import AzureOpenAI

# Ensure API credentials are set correctly
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_API_VERSION = os.getenv("AZURE_API_VERSION")

# Initialize AzureOpenAI client
client = AzureOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_API_VERSION 
) 

# Ensure the correct deployment name for embeddings
EMBEDDING_DEPLOYMENT_NAME = "text-embedding-3-large"  # Replace with your Azure deployment name

from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI

# Specify the Azure deployment name
embeddings = AzureOpenAIEmbeddings(
    model=EMBEDDING_DEPLOYMENT_NAME,  
    azure_deployment=EMBEDDING_DEPLOYMENT_NAME  # Corrected: Specify the deployment name
)

from langchain_community.vectorstores import ApertureDB

vector_db = ApertureDB.from_documents(papers, embeddings)

# Define Models
EMBEDDING_MODEL = "text-embedding-3-large"  # Ensure this matches your Azure deployment name
GENERATION_MODEL = "gpt-4"




Progress:   0%|                                | 0.00/39.0 [00:08<?, ?batches/s]


In [35]:
from smolagents import Tool

class RetrieverTool(Tool):
    name = "retriever"
    description = "Uses semantic search to retrieve documents that could be relevant to answer the query."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be semantically close to the target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "string"

    def __init__(self, client, **kwargs):
        super().__init__(**kwargs)
        self.embedder = client

    def simple_retriever(self, query: str, n=5):
        """Retrieve documents using similarity search"""
        retriever = vector_db.as_retriever(search_type="mmr", search_kwargs={"k": n})
        results = retriever.invoke(query)

        return "\nRetrieved documents:\n" + "".join(
            [
                f"\n\n===== Document {str(i)} =====\n" + doc.page_content
                for i, doc in enumerate(results)
            ]
        )

    def get_embedding(self, text: str):
        """Fix: Use correct embedding call"""
        response = self.embedder.embeddings.create(
            input=[text],  # Embeddings expect a list of strings
            engine=EMBEDDING_DEPLOYMENT_NAME  # Use Azure deployment name
        )
        return response["data"][0]["embedding"]

    def forward(self, query: str) -> str:
        assert isinstance(query, str)
        return self.simple_retriever(query)

retriever_tool = RetrieverTool(client)


In [48]:
# Initialize LLM and Agent
from smolagents import ToolCallingAgent, LiteLLMModel
import os
from smolagents import AzureOpenAIServerModel

model = AzureOpenAIServerModel(
    model_id = GENERATION_MODEL,
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    api_version="2024-08-01-preview"    
)
            
agent = ToolCallingAgent(tools=[retriever_tool], model=model)

question = "Why is calculating Higgs Boson decay important?"
agent_output = agent.run(question)
print(agent_output)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Why is calculating Higgs Boson decay important?                                                                 │
│                                                                                                                 │
╰─ AzureOpenAIServerModel - gpt-4 ────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'retriever' with arguments: {'query': 'importance of calculating Higgs Boson decay'}              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: Retrieved documents:


===== Document 0 =====
GeV. We impose the kinemati seletion QT < Q, but its in(cid:29)uene is not5 GeV, respetively. The average values of
QT are 26 and 23 GeV, omputed over the rangeDi(cid:27)erenes in the shapes of these QT spetra an be attributed to 
the di(cid:27)erent struturegg+gqS initial state, whereas the ontinuum is ontrolled primarily by the q¯q+qg initial
state. (k) (k) g = q Beause the dominant Sudakov oe(cid:30)ient A (CA/CF) in the gg ase, the resummed q¯q +qg 
initial-state radiation produes narrower QT distributions than gg + gqS initial-state radiation. About 80% of the 
diphoton rate is provided by the q¯q + qg hannel, implying a narrower QT distribution of the bakground, ifCF in the
q¯q ase is smaller than A∝(k) qA(k)based on the value of Aalone.The ontinuum bakground ontribution is also enhaned 
by (cid:28)nal-state radiation in qg sattering. The QT pro(cid:28)le of the (cid:28)nal-state ollinear terms 
depends more on the isolation model (inluding Eiso T and ∆R) than on the initial-state Sudakov exponent. For the 
nominalQT distribution, diminishing its di(cid:27)erene from the Higgs boson signal distribution. More 
e(cid:27)etive isolation may redue the impat of the (cid:28)nal-state radiation on QT distributions.the azimuthal 
angle of the photons, suh as the angle ϕ∗ in the Collins-Soper frame shownover all QT , as in Fig. 15, or 
integrated above some minimal QT value, as in an experimental measurement. Without isolation imposed, the spin-0 
Higgs boson signal must be (cid:29)at in ϕ∗, but the QCD bakground peaks toward ϕ∗ = 0 and π (i.e., sinϕ∗ = 0) as a
result of the (cid:28)nal-state qg singularity. sinϕ∗ < sin∆R. The result is a signal distribution with a broad 
peak near ϕ∗ = π/2, while the bakground favors values of ϕ∗ near 0 and π. A seletion of events with ϕ∗ in the 
viinity of π/2, and QT large enough, helps to redue the impat of the qg bakground.In the lab ,where ϕiT is the 
azimuthal angle ϕ4T| ~QT . The signal (bakground) proesses tend to have more events with large2, 3Isolation uts 
suppress both the signal and the bakground forϕ3T −frame, a related distribution is in the variable | between ~pγiT
andBy de(cid:28)nition, the reoil parton 5 always lies in the Oxz plane (has zero azimuthal angle) in the 
Collins-with 5, i.e., have sinϕ∗ = 0. One of the resummed struture funtions for the gg bakground is modulated by 
cos2ϕ∗ (see Se. II B),ϕ3T − (small) magnitude of | the rapidities ∆y = yhard −ϕ4T|. ysoft of the photons with 
harder and softer values of pγA third potential disriminator between the signal and bakground is the 
di(cid:27)erene inalmost (cid:29)at over a wide range of ∆y. Di(cid:27)erent spin orrelations in the deay of a 
spin-0singularity at ∆y 2 present in the (cid:28)nite-order ross setion and obvious in Fig. 10 of Ref. |10℄, where 
the distribution with respet to y∗ ∆y/2 is shown. The disontinuity in dσ/dQT aused by the (cid:28)nite-order 
approximation is resummed in our alulation, yielding a≈≡tanh(∆y/2) = cosθ∗ when QT is zero. The cosθ∗ distribution 
is shown in the lower-rightA omparison of QT distributions in the top-left panel of Fig. 15 suggests that the 
signal versus bakground ratio would be enhaned if a ut is made to restrit QT > 10 GeV.QT , ϕ∗, and cosθ∗ 
distributions are good disriminators between the Higgs boson signal andsetion dσ/(dQdQTdydΩ∗) as a funtion of the 
mass Q, transverse momentum QT , and rapidity y of the diphoton system, and of the polar and azimuthal angles of 
the individualomputed at next-to-leading order (NLO) in the strong oupling strength αs, and we inlude the 
state-of-art resummation of initial-state gluon radiation to all orders in αs, valid to next-reliable alulation of 
the QT dependene in the region of small and intermediate values of QT , where the ross setion is greatest. It is 
also needed for stable estimates of the e(cid:27)ets ofT in thethe region QT . Q. We show that our results at the 
Teva

[Step 1: Duration 6.52 seconds| Input tokens: 1,008 | Output tokens: 21]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'Calculating the decay of the Higgs Boson is important  │
│ due to several reasons not explicitly detailed in the retrieved documents. However, generally, the importance   │
│ can be inferred from the discussions around the precision calculations and experimental measurements related to │
│ the Higgs Boson decay. These calculations are crucial for testing the predictions of the Standard Model of      │
│ particle physics, refining our understanding of fundamental particles and forces, and potentially uncovering    │
│ new physics beyond the Standard Model. The decay patterns and rates of the Higgs Boson can provide insights     │
│ into the Higgs mechanism, which is responsible for particles acquiring mass. Moreover, discrepancies between    │
│ theoretical predictions and experimental results could indicate the existence of unknown particles or forces,   │
│ opening avenues for new research in particle physics.'}                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: Calculating the decay of the Higgs Boson is important due to several reasons not explicitly detailed 
in the retrieved documents. However, generally, the importance can be inferred from the discussions around the 
precision calculations and experimental measurements related to the Higgs Boson decay. These calculations are 
crucial for testing the predictions of the Standard Model of particle physics, refining our understanding of 
fundamental particles and forces, and potentially uncovering new physics beyond the Standard Model. The decay 
patterns and rates of the Higgs Boson can provide insights into the Higgs mechanism, which is responsible for 
particles acquiring mass. Moreover, discrepancies between theoretical predictions and experimental results could 
indicate the existence of unknown particles or forces, opening avenues for new research in particle physics.

[Step 2: Duration 4.94 seconds| Input tokens: 10,973 | Output tokens: 176]

Calculating the decay of the Higgs Boson is important due to several reasons not explicitly detailed in the retrieved documents. However, generally, the importance can be inferred from the discussions around the precision calculations and experimental measurements related to the Higgs Boson decay. These calculations are crucial for testing the predictions of the Standard Model of particle physics, refining our understanding of fundamental particles and forces, and potentially uncovering new physics beyond the Standard Model. The decay patterns and rates of the Higgs Boson can provide insights into the Higgs mechanism, which is responsible for particles acquiring mass. Moreover, discrepancies between theoretical predictions and experimental results could indicate the existence of unknown particles or forces, opening avenues for new research in particle physics.
